In [ ]:
# 安装所需的库 (qiskit和pyquafu)
# 已安装则可跳过
! pip install qiskit
! pip install pyquafu
! pip install pylatexenc
! pip install qiskit2quafu

In [ ]:
# 导入所需的库
from qiskit import QuantumCircuit
from qiskit.quantum_info import Operator
from qiskit2quafu import qiskit2quafu
from quafu import User, Task

In [ ]:
# 使用qiskit创建一个电路
qiskitCircuit = QuantumCircuit(4)
qiskitCircuit.h([0,1,2,3])
qiskitCircuit.mcx([0,1,2],3)
qiskitCircuit.ry(0.6,0)
permute = Operator([[0, 0, 1, 0, 0, 0, 0, 0],
                    [1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 0, 0, 1],
                    [0, 0, 0, 0, 0, 1, 0, 0],
                    [0, 0, 0, 0, 0, 0, 1, 0],
                    [0, 0, 0, 0, 1, 0, 0, 0]])
qiskitCircuit.unitary(permute, [0,1,3], label='P')
# 显示电路
qiskitCircuit.draw(output='mpl')

In [ ]:
# 从MQTBench_qasm载入基准线路
# 测试库中文件最后的数字表示电路位数，机器支持比特数需要大于电路位数
file = 'MQTBench_qasm/qpeexact_indep_qiskit_5.qasm'
qiskit_qc = QuantumCircuit.from_qasm_file(file)
qiskit_qc.draw(output='mpl')

In [ ]:
# 进行转译并返回两个对象: quafu电路和转译后的电路字符串
quafu_gates = ['cx','cy','cz','cp','u1','u2','u3','h','id','swap','cswap','p','rx','ry',
               'rz','x','y','z','s','sdg','t','tdg','sx','ccx','rxx','ryy','rzz']
quafuCircuit, transpiled_qasm = qiskit2quafu(qiskitCircuit, basis_gates=quafu_gates)
# 显示转译后的电路
quafuCircuit.draw_circuit()

In [ ]:
# 电路准备完成，接下来设置quafu用户信息
# 进入 http://quafu.baqis.ac.cn 注册

# 注册并登录后, 在主页右上部分找到API Token
# 复制并在这里填入APi Token
api_token='YTlsFPF0wZ50RDCwn1L1BbE5rmpQdXn21XtEI55po_n.Qf5ATO3ETO1kjNxojIwhXZiwCMxMjM6ICZpJye.9JiN1IzUIJiOicGbhJCLiQ1VKJiOiAXe0Jye'

# 填入想使用的机器
# 可选 "ScQ-P10", "ScQ-P18", "ScQ-P136"
backend = "ScQ-P136"

# 创建用户对象
user = User()
user.save_apitoken(api_token)

In [ ]:
# 上传任务并获取任务编号
task = Task()
task.config(backend=backend, shots=3000, compile=True, priority=2)
task_id = task.send(quafuCircuit, wait=False, name='', group='g').taskid
print ('The task id is:',task_id)

In [ ]:
# 获取任务状态, 完成则显示结果
# 如果任务未完成, 稍等片刻之后重新执行这个单元格即可
task_status = task.retrieve(task_id).task_status
if task_status == 'In Queue' or task_status == 'Running':
    print("The current task status is '%s', please wait." % task_status)
elif task_status == 'Completed':
    print("The task execution has completed and the result has been returned.")
    res = task.retrieve(task_id)
    res.plot_probabilities()
    
    
'''
['cx','cy','cz','cp','u1','u2','u3','h','id','swap','cswap','p','rx','ry',
               'rz','x','y','z','s','sdg','t','tdg','sx','ccx','rxx','ryy','rzz']
'''